# FIKH Model: Large Amplitude Oscillatory Shear (LAOS)

## Learning Objectives

1. Fit FIKH to **LAOS (large amplitude oscillatory)** experimental data
2. Analyze **Lissajous curves** (stress-strain loops) with fractional memory
3. Extract **Fourier harmonics** and understand nonlinear contributions
4. Explore how **alpha_structure** affects intra-cycle structure evolution
5. Quantify yielding and thixotropy during oscillatory flow

## Prerequisites

- NB01: FIKH Flow Curve (concepts)
- NB05: FIKH SAOS (linear viscoelasticity)
- Bayesian inference fundamentals

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~5-8 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~20-30 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FIKH

sys.path.insert(0, os.path.join("..", "utils"))
from fikh_tutorial_utils import (
    load_pnas_laos,
    save_fikh_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
    get_fikh_param_names,
    print_alpha_interpretation,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Fractional LAOS Response

In LAOS, the strain amplitude is large enough to probe **nonlinear** material response:
$$
\gamma(t) = \gamma_0 \sin(\omega t)
$$

### Nonlinear Features

1. **Lissajous curves**: Non-elliptical stress-strain loops
2. **Higher harmonics**: $\sigma(t) = \sum_n (G'_n \sin(n\omega t) + G''_n \cos(n\omega t))$
3. **Intra-cycle yielding**: Structure breakdown within each cycle

### Alpha Effect on LAOS

- **Lower alpha**: Memory effects persist across cycles, slower equilibration
- **Higher alpha**: Each cycle approaches independent behavior

### Key Indicators

- **e₃/e₁ ratio**: Measure of nonlinearity (zero for linear response)
- **Lissajous shape**: Rhomboidal (thixotropic), rectangular (yield stress)

## 3. Load Data

In [ ]:
# Load PNAS LAOS data
OMEGA = 1.0  # rad/s
STRAIN_AMP_INDEX = 5  # Medium amplitude

time_data, strain_data, stress_data = load_pnas_laos(
    omega=OMEGA,
    strain_amplitude_index=STRAIN_AMP_INDEX,
)

# Estimate strain amplitude from data
gamma_0 = (np.max(strain_data) - np.min(strain_data)) / 2

print(f"Data points: {len(time_data)}")
print(f"Time range: [{time_data.min():.4f}, {time_data.max():.2f}] s")
print(f"Strain amplitude: {gamma_0:.4f}")
print(f"Angular frequency: {OMEGA} rad/s")

In [ ]:
# Plot raw LAOS data
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Time series
ax1 = axes[0]
ax1.plot(time_data, strain_data, "b-", lw=1, alpha=0.7, label="Strain")
ax1_twin = ax1.twinx()
ax1_twin.plot(time_data, stress_data, "r-", lw=1, alpha=0.7, label="Stress")
ax1.set_xlabel("Time [s]", fontsize=12)
ax1.set_ylabel("Strain [-]", fontsize=12, color="blue")
ax1_twin.set_ylabel("Stress [Pa]", fontsize=12, color="red")
ax1.set_title("Time Series", fontsize=13)

# Lissajous curve (elastic)
ax2 = axes[1]
ax2.plot(strain_data, stress_data, "k-", lw=1)
ax2.set_xlabel("Strain [-]", fontsize=12)
ax2.set_ylabel("Stress [Pa]", fontsize=12)
ax2.set_title("Elastic Lissajous (σ vs γ)", fontsize=13)
ax2.grid(True, alpha=0.3)

# Lissajous curve (viscous)
gamma_dot_data = np.gradient(strain_data, time_data)
ax3 = axes[2]
ax3.plot(gamma_dot_data, stress_data, "k-", lw=1)
ax3.set_xlabel("Strain rate [1/s]", fontsize=12)
ax3.set_ylabel("Stress [Pa]", fontsize=12)
ax3.set_title("Viscous Lissajous (σ vs γ̇)", fontsize=13)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
# Create and fit FIKH model
model = FIKH(include_thermal=False, alpha_structure=0.7)

t0 = time.time()
model.fit(time_data, stress_data, test_mode="laos", strain=strain_data)
t_nlsq = time.time() - t0

param_names = get_fikh_param_names(include_thermal=False)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Predict LAOS response
laos_result = model.predict_laos(time_data, gamma_0=gamma_0, omega=OMEGA)
stress_pred = np.asarray(laos_result["stress"])

metrics = compute_fit_quality(stress_data, stress_pred)
print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g} Pa")

In [ ]:
# Plot fit: Lissajous curves
strain_pred = laos_result["strain"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Elastic Lissajous
ax1.plot(strain_data, stress_data, "ko", markersize=3, alpha=0.5, label="Data")
ax1.plot(strain_pred, stress_pred, "-", lw=2, color="C0", label="FIKH fit")
ax1.set_xlabel("Strain [-]", fontsize=12)
ax1.set_ylabel("Stress [Pa]", fontsize=12)
ax1.set_title(f"Elastic Lissajous (R$^2$ = {metrics['R2']:.4f})", fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Time series comparison
ax2.plot(time_data, stress_data, "ko", markersize=3, alpha=0.5, label="Data")
ax2.plot(time_data, stress_pred, "-", lw=2, color="C0", label="FIKH fit")
ax2.set_xlabel("Time [s]", fontsize=12)
ax2.set_ylabel("Stress [Pa]", fontsize=12)
ax2.set_title("Stress Time Series", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Alpha Effect on LAOS

In [ ]:
# Compare Lissajous curves for different alpha values
alpha_values = [0.3, 0.5, 0.7, 0.9, 0.99]
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(alpha_values)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

original_alpha = model.parameters.get_value("alpha_structure")

for alpha, color in zip(alpha_values, colors):
    model.parameters.set_value("alpha_structure", alpha)
    result = model.predict_laos(time_data, gamma_0=gamma_0, omega=OMEGA)
    stress_alpha = np.asarray(result["stress"])
    strain_alpha = np.asarray(result["strain"])
    
    ax1.plot(strain_alpha, stress_alpha, "-", color=color, lw=1.5, label=f"α = {alpha:.2f}")
    ax2.plot(time_data[:200], stress_alpha[:200], "-", color=color, lw=1.5, label=f"α = {alpha:.2f}")

model.parameters.set_value("alpha_structure", original_alpha)

# Add data
ax1.plot(strain_data, stress_data, "ko", markersize=2, alpha=0.3, label="Data")
ax2.plot(time_data[:200], stress_data[:200], "ko", markersize=2, alpha=0.3, label="Data")

ax1.set_xlabel("Strain [-]", fontsize=12)
ax1.set_ylabel("Stress [Pa]", fontsize=12)
ax1.set_title("Lissajous Curves vs Alpha", fontsize=13)
ax1.legend(fontsize=8, loc="best")
ax1.grid(True, alpha=0.3)

ax2.set_xlabel("Time [s]", fontsize=12)
ax2.set_ylabel("Stress [Pa]", fontsize=12)
ax2.set_title("First Cycles: Stress vs Time", fontsize=13)
ax2.legend(fontsize=8, loc="best")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Physical interpretation
fitted_alpha = model.parameters.get_value("alpha_structure")
print_alpha_interpretation(fitted_alpha)

## 6. Fourier Harmonic Analysis

In [ ]:
# Extract Fourier harmonics from LAOS response
def extract_harmonics(time, stress, omega, n_harmonics=5):
    """Extract Fourier harmonics from LAOS stress response."""
    period = 2 * np.pi / omega
    n_cycles = int(time[-1] / period)
    
    # Use last 2 cycles for steady-state
    if n_cycles >= 2:
        start_idx = int(len(time) * (n_cycles - 2) / n_cycles)
    else:
        start_idx = 0
    
    t_ss = time[start_idx:]
    stress_ss = stress[start_idx:]
    
    harmonics = {}
    for n in range(1, n_harmonics + 1):
        # Integrate: e_n = (2/T) ∫ σ(t) sin(nωt) dt
        # v_n = (2/T) ∫ σ(t) cos(nωt) dt
        sin_term = np.sin(n * omega * t_ss)
        cos_term = np.cos(n * omega * t_ss)
        
        e_n = 2 * np.trapezoid(stress_ss * sin_term, t_ss) / (t_ss[-1] - t_ss[0])
        v_n = 2 * np.trapezoid(stress_ss * cos_term, t_ss) / (t_ss[-1] - t_ss[0])
        
        harmonics[f"e_{n}"] = e_n
        harmonics[f"v_{n}"] = v_n
    
    return harmonics

# Extract harmonics from data and fit
harmonics_data = extract_harmonics(time_data, stress_data, OMEGA)
harmonics_fit = extract_harmonics(time_data, stress_pred, OMEGA)

print("Fourier Harmonic Comparison:")
print("=" * 50)
print(f"{'Harmonic':>10s}  {'Data':>12s}  {'FIKH':>12s}")
print("-" * 50)
for key in harmonics_data:
    print(f"{key:>10s}  {harmonics_data[key]:12.4g}  {harmonics_fit[key]:12.4g}")

# Nonlinearity indicator
e3_e1_data = abs(harmonics_data["e_3"]) / (abs(harmonics_data["e_1"]) + 1e-10)
e3_e1_fit = abs(harmonics_fit["e_3"]) / (abs(harmonics_fit["e_1"]) + 1e-10)
print(f"\nNonlinearity (|e₃/e₁|): Data = {e3_e1_data:.4f}, FIKH = {e3_e1_fit:.4f}")

## 7. Bayesian Inference

In [ ]:
# Bayesian inference
initial_values = {name: model.parameters.get_value(name) for name in param_names}

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    time_data,
    stress_data,
    test_mode="laos",
    strain=strain_data,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence
all_pass = print_convergence_summary(result, param_names)

In [ ]:
# Parameter comparison
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, param_names)

## 8. Save Results

In [ ]:
save_fikh_results(model, result, "fikh", "laos", param_names)
print("\nResults saved.")

## Key Takeaways

1. **LAOS reveals nonlinear viscoelastic response** beyond SAOS regime
2. **Lissajous curves** show intra-cycle yielding and structure evolution
3. **Lower alpha** → memory persists across cycles, slower equilibration
4. **Higher alpha** → each cycle more independent (classical behavior)
5. **Fourier harmonics (e₃/e₁)** quantify nonlinearity
6. **LAOS data constrains yield and thixotropy** parameters together

### FIKH Tutorial Series Complete!

You have now completed all 6 FIKH tutorials:
- **NB01**: Flow curve (steady-state, alpha introduction)
- **NB02**: Startup shear (stress overshoot, transient)
- **NB03**: Stress relaxation (power-law tails)
- **NB04**: Creep (delayed yielding)
- **NB05**: SAOS (linear viscoelasticity)
- **NB06**: LAOS (nonlinear oscillatory)

### Next: FMLIKH Multi-Mode Tutorials

- **NB07-NB12**: FMLIKH (multi-mode fractional model) tutorials